<a href="https://colab.research.google.com/github/thainehassumpcao/Up2Date/blob/master/Up2Date_prototype_checkpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Up2Date prototype**
This prototype showcases the concept proposed by our team (Up2Date) at the Copernicus Barcelona Hackathon.

**Goal:** to improve flood preparedness by updating GloFAS impact forecast services, based on satellite images. The Minimum Viable Product is focused on agricultural areas.

Two products are proposed:

*   Product 1: updated affected agricultural areas
*   Product 2: updated impact assessment statistics


### **Prototype details and methodology**

We focus on a flood occuring in September 2020 at the Sindh province in Pakistan, namely updating the forecast done on 01-09-2020.

The steps taken are:

1.   Fetch forecasted data:
    * Flood extent: manifactured to be similar to the flood extent visualized in GloFAS. It was created based on the Copernicus global flood hazard map with return of 10 years. 
    * Impact assessment: extracted manually from GloFAS html
2.   Search satellite images
3.   





In [ ]:
import numpy as np
import requests
from zipfile import ZipFile
from pathlib import Path
import pandas as pd
from bs4 import BeautifulSoup

##Fetch forecasted data

In [ ]:
# Fetch flood extent map
def fetch_static_flood_map(rp, location=None):
    """ Fetch static flood hazard maps from Copernicus"""

    filename = 'floodMapGL_rp{}y.zip'.format(rp)
    url = 'http://cidportal.jrc.ec.europa.eu/' \
    'ftp/jrc-opendata/FLOODS/GlobalMaps/{}'.format(filename)

    r = requests.get(url, allow_redirects=True)
    open(filename, 'wb').write(r.content)
    with ZipFile(filename, 'r') as zipObj:
        zipObj.extractall()
    Path(filename).unlink()

def load_toy_flood_extent_map():
    """Load a .tiff flood extent map created for this prototype"""

    if Path('AOI.geojson').is_file():
        print('File already exists')
        return
    else:
        !git clone -l -s https://github.com/thainehassumpcao/Up2Date cloned-repo
        return

# fetch_static_flood_map(100)
load_toy_flood_extent_map()

Cloning into 'cloned-repo'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 10 (delta 3), reused 6 (delta 1), pack-reused 0
Unpacking objects: 100% (10/10), done.


In [ ]:
# Fetch impact assessment 
html_string = """
<tbody>
    <tr>
        <th scope="row">Estimated protection levels [yr]</th>
        <td class="text-center">0</td>
    </tr>
    <tr>
        <th scope="row">Population affected [No. of people]</th>
        <td class="text-center">325200</td>
    </tr>
    <tr>
        <th scope="row">Artificial surfaces [ha]</th>
        <td class="text-center">1</td>
    </tr>
    <tr>
        <th scope="row">Agricultural surfaces [ha]</th>
        <td class="text-center">914</td>
    </tr>
    <tr>
        <th scope="row">Forest and seminatural [ha]</th>
        <td class="text-center">N/A</td>
    </tr>
    <tr>
        <th scope="row">Estimated peak time [d]</th>
        <td class="text-center">1</td>
    </tr>
    <tr>
        <th scope="row">Estimated mean return period [yr]</th>
        <td class="text-center">12</td>
    </tr>
    <tr>
        <th scope="row">Mean probability of exceeding 2-years threshold</th>
        <td class="text-center">100</td>
    </tr>
    <tr>
        <th scope="row">Mean probability of exceeding 5-years threshold</th>
        <td class="text-center">100</td>
    </tr>
    <tr>
        <th scope="row">Mean probability of exceeding 20-years threshold</th>
        <td class="text-center">24</td>
    </tr>
</tbody>
"""

soup = BeautifulSoup(html_string, 'lxml')
table = soup.find_all('tbody')[0]
n_rows = len(table.find_all('th'))
new_table = pd.DataFrame(columns=range(0,2), index = range(0,n_rows))

for row_number, row in enumerate(table.find_all('tr')):
    column_parameter = row.find_all('th')
    column_value = row.find_all('td')
    new_table.iat[row_number, 0] = column_parameter[0].get_text()
    new_table.iat[row_number, 1] = column_value[0].get_text()

new_table

,0,1
0,Estimated protection levels [yr],0
1,Population affected [No. of people],325200
2,Artificial surfaces [ha],1
3,Agricultural surfaces [ha],914
4,Forest and seminatural [ha],N/A
5,Estimated peak time [d],1
6,Estimated mean return period [yr],12
7,Mean probability of exceeding 2-years threshold,100
8,Mean probability of exceeding 5-years threshold,100
9,Mean probability of exceeding 20-years threshold,24


##Search satellite images

In [ ]:
# Prepare Area of Interest Polygon
#Extract max extents from shapefile

In [ ]:
# Prepare the API - Require credentials to access the copernicus hub:
user = input('User name: ')
password = getpass('Password: ')
api = SentinelAPI(user, password, 'https://scihub.copernicus.eu/apihub')
del user, password

# Query products based on an Area of Interest and date interval
AOI = read_geojson('AOI.geojson')
products = api.query(
    area = geojson_to_wkt(AOI),
    date = ('20200523', '20200627'),
    platformname = 'Sentinel-2',
    cloudcoverpercentage=(0,30))

##Assess crop yield

#Re-assess impact

## Product 1: Affected agricultural areas

## Product 2: updated impact assessment